## Data Visualization

Import relevant packages

In [12]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import pandas as pd
import mysql.connector
import re
from nltk.stem import PorterStemmer
import geopandas as gpd
import folium
from ipyleaflet import Map, Marker, basemaps, LayerGroup, LayersControl, AwesomeIcon, WidgetControl
import ipywidgets as widgets
from ipywidgets import widgets, Button, VBox, Layout
from IPython.display import display


function to load a given table from my local database

In [13]:
def load(table):
    mycursor.execute("SELECT * FROM " + table)
    df = pd.DataFrame(mycursor.fetchall())
    df.columns = [i[0] for i in mycursor.description]
    return df

In [14]:
# Establishing connection to the database
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="free_palestine",
    database="mamluk_jerusalem"
)

mycursor = mydb.cursor()

# Defining the path to the data files
PATH = "D:\MA\MA2\Semester_Project\data"

# Loading the GeoDataFrame from the GeoJSON file
gdf = gpd.read_file(PATH + "/mamluk_jerusalem_buildings_location.geojson")

# Converting the coordinate reference system to EPSG:4326 (WGS84)
gdf = gdf.to_crs(epsg=4326)

# Extracting the longitude and latitude values from the geometry
gdf["longitude"] = gdf["geometry"].apply(lambda x: x.x)
gdf["latitude"] = gdf["geometry"].apply(lambda x: x.y)

# Dropping the geometry column from the GeoDataFrame
gdf = gdf.drop("geometry", axis=1)

# Loading the dataframes from the files
building_df = load("building")
date_df = load("date")
arch_df = load("architecture")
hist_df = load("history")
insc_df = load("inscription")

This function takes a list of paragraphs (text) and a subtitle string. It searches for a paragraph that starts with the same words as the subtitle and returns the concatenated text from the paragraph following the matching paragraph until the next paragraph starting with uppercase letters

In [15]:
def single_row_subtitle_parag(text, subtitle):
    if subtitle == None:
        return None
    subtitle_length = len(subtitle.split(" "))
    append = False
    end = len(text)  # Set default value for end
    for j, paragraph in enumerate(text):
        
        first_word = paragraph.strip().replace("\n", " ")
        first_word = first_word.split(" ")[0:subtitle_length]
        #if first_word different from subtitle, continue
        #subtitle can be two words

        if not all(a == b for a, b in zip(first_word, subtitle.split(" "))):
            continue
        
        append = True
        start = j+1
        for k in range(j+2, len(text)):
            next_first_word = text[k].replace("\n", " ").split(" ")[0]
            if next_first_word.isupper() and len(re.sub(r'[^a-zA-Z]', '', next_first_word)) > 3:
                end = k
                break
    
    if append:
        return " ".join(text[start:end])


Map creation

In [16]:
# Create the map centered in Jerusalem
location = [gdf["latitude"].mean(), gdf["longitude"].mean()]
m = Map(center=location, zoom=15, basemap=basemaps.OpenStreetMap.Mapnik)

# Create widgets for the side panel
pane = widgets.Tab()
pane_layout = Layout(overflow='auto', height='370px', width='300px')  # Set fixed height and width for the pane
# Set the pane's layout
pane.layout = pane_layout
general_content_widget = widgets.HTML(layout=Layout(overflow='auto', height='100%'))  # Make content scrollable
date_content_widget = widgets.HTML(layout=Layout(overflow='auto', height='100%'))  # Make content scrollable
history_content_widget = widgets.HTML(layout=Layout(overflow='auto', height='100%'))  # Make content scrollable
architecture_content_widget = widgets.HTML(layout=Layout(overflow='auto', height='100%'))  # Make content scrollable

dropdown_layout = Layout(width='90%')  # Reduce dropdown width to 80% of the pane's width
history_dropdown = widgets.Dropdown(description='Subtitles:', disabled=False, layout=dropdown_layout)
architecture_dropdown = widgets.Dropdown(description='Subtitles:', disabled=False, layout=dropdown_layout)

# Create a container for the dropdown and the content widget for each tab
history_container = widgets.VBox([history_dropdown, history_content_widget])
architecture_container = widgets.VBox([architecture_dropdown, architecture_content_widget])

pane.children = [general_content_widget, date_content_widget, history_container, architecture_container]

# Create a close button
close_button = Button(description="X", button_style='danger', layout=Layout(width='auto', height='auto'))
close_button.layout.margin = '0 0 0 auto'  # Position button to the right

# Create a container for the pane and the close button
pane_container = VBox([close_button, pane])
# Create a container for the map and the pane
box = widgets.HBox([m])

def close_pane(button):
    # Hide the pane when the close button is clicked
    box.children = [m]

close_button.callbacks = []
# Register the callback for the close button
close_button.on_click(close_pane)

def make_on_marker_click(marker):
    def on_marker_click(*args, **kwargs):        
        general_content, date_content, history_content, architecture_content, history_sub, arch_sub, hist_text, arch_text = marker.content
        general_content_widget.value = general_content
        date_content_widget.value = date_content
        default_message = '<br><div style="text-align: center;"><b>Select a topic you want to read about :)</b></div>'
        # Set the content widgets for the history and architecture tabs
        history_content_widget.value = default_message if history_content == '' else history_content
        architecture_content_widget.value = default_message if architecture_content == '' else architecture_content
        

        # Reset the dropdowns
        history_dropdown.value = None
        architecture_dropdown.value = None

        # Shorten the dropdown options and add tooltips
        history_dropdown.options = [(option[:20] + '...' if len(option) > 20 else option, option) for option in history_sub]
        architecture_dropdown.options = [(option[:20] + '...' if len(option) > 20 else option, option) for option in arch_sub]

        # Set the dropdown event handlers
        history_dropdown.observe(hist_dropdown_eventhandler(hist_text, history_content_widget), names='value')
        architecture_dropdown.observe(arch_dropdown_eventhandler(arch_text, architecture_content_widget), names='value')

        pane.selected_index = 0  # Select the "General" tab
        box.children = [m, pane_container]  # Show the pane when a marker is clicked
    return on_marker_click

def hist_dropdown_eventhandler(hist_text, history_content_widget):
    def eventhandler(change):
        hist_new_content = single_row_subtitle_parag(hist_text, change.new)
        # Set the content to the new content
        if hist_new_content is not None:
            history_content_widget.value = truncate_text(hist_new_content)
        else:
            history_content_widget.value = ''
    return eventhandler


def arch_dropdown_eventhandler(arch_text, architecture_content_widget):
    def eventhandler(change):
        arch_new_content = single_row_subtitle_parag(arch_text, change.new)
        # Set the content to the new content
        if arch_new_content is not None:
            architecture_content_widget.value = truncate_text(arch_new_content)
        else:
            architecture_content_widget.value = ''
    return eventhandler
        
    
def truncate_text(text, limit=100):
    # Truncate text to limit characters, adding a 'Show more' button if it's too long
    if len(text) > limit:
        escaped_text = text.replace('\\', '\\\\').replace("'", "\\'").replace('"', '\\"').replace('\n', '\\n')
        return text[:limit] + '... <button onclick="this.parentElement.innerHTML=\'' + escaped_text + '\'">Show more</button>'
    else:
        return text

# Create a separate pane for inscriptions
inscription_content_widget = widgets.HTML(layout=Layout(overflow='auto', height='100%'))  # Make content scrollable
#add a margin around the content widget
inscription_content_widget.layout.margin = '10px'

inscription_pane = widgets.VBox([inscription_content_widget])
inscription_pane.layout = Layout(overflow='auto', height='350px', width='300px')
inscription_close_button = Button(description="X", button_style='danger', layout=Layout(width='auto', height='auto'))
inscription_close_button.layout.margin = '0 0 0 auto'  # Position button to the right


# Add a title to the inscription pane
inscription_title_label = widgets.HTML(value='<b>INSCRIPTIONS IN </b>', layout=Layout(width='auto', height='auto'))
inscription_title_label.layout.margin = '0 0 0 10px'  # Add margin to the left

# Create a container for the title label and the close button
inscription_pane_header = VBox([close_button, inscription_title_label], layout=Layout(align_items='center'))
inscription_pane_container = VBox([inscription_pane_header, inscription_pane])

def close_inscription_pane(button):
    # Hide the pane when the close button is clicked
    box.children = [m]

# Register the callback for the close button
inscription_close_button.callbacks = []
inscription_close_button.on_click(close_inscription_pane)

def make_on_inscription_marker_click(marker, name):
    def on_inscription_marker_click(*args, **kwargs):
        inscription_title_label.value = '<b>INSCRIPTIONS IN ' + name + '</b>'
        inscription_title_label.layout.margin = '0 auto 0 auto'
        previous_content = inscription_content_widget.value
        new_content = marker.content
        inscription_content_widget.value =  new_content
        box.children = [m, inscription_pane_container]  # Show the inscription pane when a marker is clicked
    return on_inscription_marker_click

#split text into paragraphs: each paragraph starts and ends with \n\n
arch_df["Text"] = arch_df["others"].apply(lambda x: x.split("\n\n"))
hist_df["Text"] = hist_df["others"].apply(lambda x: x.split("\n\n"))

#drop others = nan
arch_df = arch_df[arch_df["Text"] != "nan"]
arch_df = arch_df.reset_index(drop=True)
hist_df = hist_df[hist_df["Text"] != "nan"]
hist_df = hist_df.reset_index(drop=True)

In [17]:
# Add a LayerGroup for each group of markers
building_layer_group = LayerGroup(name='Buildings')
inscription_layer_group = LayerGroup(name='Inscriptions', visible=False)

# Add LayersControl to switch between the groups
control = LayersControl(position='bottomleft', collapsed=False, autoZIndex=False)
m.add_control(control)


In [18]:
# Iterate over the rows of the dataframe
markers = []
for i, row in gdf.iterrows():
    # Your code to populate rows...
    building_row = building_df[building_df["id"] == i+1]
    date_row = date_df[date_df["id"] == i+1]
    arch_row = arch_df[arch_df["id"] == i+1]
    hist_row = hist_df[hist_df["id"] == i+1]
    arch_sub = []
    hist_sub = []

    #split hist_row into paragraphs with \n\n
    if hist_row.empty:
        hist_text = []
    else:
        hist_text = hist_row["others"].values[0].split("\n\n")

    if arch_row.empty:
        arch_text = []
    else:
        arch_text = arch_row["others"].values[0].split("\n\n")

    #if modern_name or type are None, replace with "unknown"
    if building_row["modern_name"].values[0] is None:
        building_row.loc[i, "modern_name"] = "unknown"
    if building_row["type"].values[0] is None:
        building_row.loc[i, "type"] = "unknown"

    if date_row.empty:
        date_row = pd.DataFrame([
            {"id": i+1, "hijri": "unknown", "gregorian": "unknown", "explanation": "unknown"}
        ])
    if arch_row.empty:
        arch_row = pd.DataFrame([
            {"id": i+1, "others": "unknown"}
        ])
    else:
        sub = str(arch_row["subtitles"].values[0]).split(",")
        if sub[0] != '':
            arch_sub = [x.strip() for x in sub if x != '']              
        
    if hist_row.empty:
        hist_row = pd.DataFrame([
            {"id": i+1, "identification": "unknown", "founder": "unknown", "endowment": "unknown", "subseq_hist": "unknown"}
        ])
    else:
        sub = str(hist_row["subtitles"].values[0]).split(",")
        if sub[0] != '':
            hist_sub = [x.strip() for x in sub]
                

    general_content = '''
        <div>
            <b>Name:</b> {name}<br>
            <b>Modern name:</b> {modern_name}<br>
            <b>Use:</b> {type}<br>
        </div>
    '''.format(name=building_row["name"].values[0], 
               modern_name=building_row["modern_name"].values[0], 
               type=building_row["type"].values[0])
    
    date_content = '''
        <div>
            <b>Hijri:</b> {hijri}<br>
            <b>Gregorian:</b> {gregorian}<br>
            <b>Explanation:</b> {explanation}<br>
        </div>
    '''.format(hijri=date_row["hijri"].values[0],
                gregorian=date_row["gregorian"].values[0],
                explanation=date_row["explanation"].values[0])
    
    history_content = ''
    architecture_content = ''
        
    # Create a marker for each location
    marker = Marker(location=[row["latitude"], row["longitude"]], draggable=False, title=str(int(row['id'])))
    
    # Set the content for the marker
    marker.content = (general_content, date_content, history_content, architecture_content, hist_sub, arch_sub, hist_text, arch_text)
    markers.append(marker)
    # Register the callback function for marker click events
    marker.on_click(make_on_marker_click(marker))

    # Add the marker to the map
    building_layer_group.add_layer(marker)
    
# Add the group of building markers to the map
m.add_layer(building_layer_group)


In [19]:
# Iterate over the rows of insc_df to create new markers
for i, row in insc_df.iterrows():
    # Extract all inscriptions for the current building
    inscriptions = insc_df[insc_df["building_id"] == row["building_id"]]

    
    # Prepare the content to display when clicking on the marker
    inscription_content = '<div style="margin: 10px;">' + '<br>'.join([
        truncate_text(f'<b>{ref}:</b>\n {text}')
        for ref, text in zip(inscriptions["reference"], inscriptions["text"])
    ]) + '</div>'
    #extract the latitude from gdf
    latitude = gdf[gdf["id"] == row["building_id"]]["latitude"].values[0]
    longitude = gdf[gdf["id"] == row["building_id"]]["longitude"].values[0]

    # Create a marker for each unique building_id
    inscription_icon = AwesomeIcon(name="info", marker_color='red', icon_color='white')
    inscription_marker = Marker(location=[latitude, longitude], draggable=False, icon=inscription_icon, title=str(row['building_id']))

    # Set the content for the marker
    inscription_marker.content = inscription_content
    building_name = building_df[building_df["id"] == row["building_id"]]["name"].values[0]
    # Register the callback function for marker click events
    inscription_marker.on_click(make_on_inscription_marker_click(inscription_marker, building_name))

    # Add the marker to the inscription layer
    inscription_layer_group.add_layer(inscription_marker)

# Add the group of inscription markers to the map
m.add_layer(inscription_layer_group)

In [20]:
# Create a list of tuples with building names and DataFrame indices
options = [(name, i) for i, name in enumerate(building_df["name"])]

search_bar = widgets.Dropdown(options=options, description='Search:')
search_bar.layout.width = '250px'  # adjust the width as needed

highlighted_marker = None  # This will keep track of the currently highlighted marker

def on_value_change(change):
    global highlighted_marker

    # If there is a previously highlighted marker, reset its icon
    if highlighted_marker is not None:
        highlighted_marker.icon = AwesomeIcon(name="map-marker", marker_color='blue', icon_color='black')

    # Get the DataFrame index of the selected building
    idx = change['new']

    # Highlight the selected marker by changing its icon
    highlighted_marker = markers[idx]
    highlighted_marker.icon = AwesomeIcon(name="map-marker", marker_color='orange', icon_color='white')
            
    
    # Extract latitude and longitude values
    lat = gdf.loc[idx, "latitude"]
    lon = gdf.loc[idx, "longitude"]
    
    # Center the map on the selected building and zoom in
    m.center = (lat, lon)
    m.zoom = 19  # Adjust the zoom level as needed

# Attach the handler to the value change event
search_bar.observe(on_value_change, names='value')

# Add search_bar to the map using a WidgetControl
widget_control = WidgetControl(widget=search_bar, position='topright')
m.add_control(widget_control)


In [21]:
# Set the titles of the Tab widget
pane.set_title(0, 'General')
pane.set_title(1, 'Date')
pane.set_title(2, 'History')
pane.set_title(3, 'Architecture')
# Display the map and content container
display(box)